In [53]:
import ipywidgets as widgets
from ipywidgets import Checkbox, VBox
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

df = vaex.open("../hdf5_pinIO/test8/pin_out.h5")
df
namesFromFile = ["Hardcoded Tag 1", "2", "Three", "4444"]
tagsFromFile = [0, 1, 2, 3]
numTags = len(namesFromFile)

df['index'] = np.arange(0, df.ADDRESS.count())

for tag in tagsFromFile:
    df.select(df.tag == tag, mode='or')


def updateGraph(change):
    if(change.name == 'value'):
        
        name = change.owner.description
        tag = tagsFromFile[namesFromFile.index(name)]

        if(change.new == True):
            df.select(df.tag == tag, mode='or')
        else:
            df.select(df.tag == tag, mode='subtract')
        

items = [Checkbox(description=namesFromFile[i], value=True, disabled=False, indent=False) for i in range(numTags)]

for i in range(numTags):
    items[i].observe(updateGraph)
    
checks = VBox(items)

df.plot_widget(df.index, df.ADDRESS, selection=[True])
    
display(checks)

PlotTemplate(components={'main-widget': VBox(children=(VBox(children=(Figure(axes=[Axis(color='#666', grid_col…

In [48]:
import ipywidgets as widgets
from ipywidgets import Checkbox, VBox
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt

tagFile = vaex.open("../hdf5_pinIO/test8/pin_out.h5")
tagFile.ADDRESS.count()
df.x = range
df['test'] = 
df

#,ADDRESS,WRITE,tag,test
<i style='opacity: 0.6'>0</i>,0,0,0,0
<i style='opacity: 0.6'>1</i>,1,1,1,1
<i style='opacity: 0.6'>2</i>,2,0,2,2
<i style='opacity: 0.6'>3</i>,3,1,3,3
<i style='opacity: 0.6'>4</i>,4,0,0,4
...,...,...,...,...
"<i style='opacity: 0.6'>999,995</i>",999995,1,3,999995
"<i style='opacity: 0.6'>999,996</i>",999996,0,0,999996
"<i style='opacity: 0.6'>999,997</i>",999997,1,1,999997
"<i style='opacity: 0.6'>999,998</i>",999998,0,2,999998
